In [1]:
import pandas as pd 
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [2]:
aplFile = '/Users/swethakolalapudi/Desktop/Reg_Data/AAPL.csv'
nasdaqFile = '/Users/swethakolalapudi/Desktop/Reg_Data/NASDAQ.csv'
xomFile = '/Users/swethakolalapudi/Desktop/Reg_Data/XOM.csv'

In [3]:
apl = pd.read_csv(aplFile,sep=",",usecols=[0,6],names=['Date','apl'],header=0)
apl.head()

Date         apl
0  2017-01-03  120.826149
1  2016-12-01  115.320023
2  2016-11-01  110.042900
3  2016-10-03  112.472404
4  2016-09-01  111.987015

In [4]:
nasdaq = pd.read_csv(nasdaqFile,sep=",",usecols=[0,6],names=['Date','Nasdaq'],header=0)
apl['Nasdaq'] = nasdaq['Nasdaq']

apl.head()

Date         apl       Nasdaq
0  2017-01-03  120.826149  5614.790039
1  2016-12-01  115.320023  5383.120117
2  2016-11-01  110.042900  5323.680176
3  2016-10-03  112.472404  5189.140137
4  2016-09-01  111.987015  5312.000000

In [5]:
xom = pd.read_csv(xomFile,sep=",",usecols=[0,6],names=['Date','xom'],header=0)
apl['xom'] = xom['xom']

apl.head()

Date         apl       Nasdaq        xom
0  2017-01-03  120.826149  5614.790039  83.129852
1  2016-12-01  115.320023  5383.120117  89.442131
2  2016-11-01  110.042900  5323.680176  86.508957
3  2016-10-03  112.472404  5189.140137  81.840340
4  2016-09-01  111.987015  5312.000000  85.730011

In [6]:
apl['Date'] = pd.to_datetime(apl['Date'], format='%Y-%m-%d')

In [7]:
apl = apl.sort_values(['Date'], ascending=[True])

In [8]:
returns = apl[['apl','Nasdaq','xom']].pct_change()
returns.head()

apl    Nasdaq       xom
120       NaN       NaN       NaN
119 -0.013064 -0.019392 -0.028539
118  0.098097  0.002272  0.052595
117  0.074158  0.042719  0.052087
116  0.214329  0.031456  0.052268

In [9]:
returns = returns[1:]
returns.head()

apl    Nasdaq       xom
119 -0.013064 -0.019392 -0.028539
118  0.098097  0.002272  0.052595
117  0.074158  0.042719  0.052087
116  0.214329  0.031456  0.052268
115  0.007014 -0.000495  0.008537

In [10]:
xData = returns[['Nasdaq','xom']]
yData = returns['apl']

In [11]:
xData[:5]

Nasdaq       xom
119 -0.019392 -0.028539
118  0.002272  0.052595
117  0.042719  0.052087
116  0.031456  0.052268
115 -0.000495  0.008537

In [12]:
yData[:5]

119   -0.013064
118    0.098097
117    0.074158
116    0.214329
115    0.007014
Name: apl, dtype: float64

In [13]:
goodAplModel = LinearRegression()
goodAplModel.fit(xData,yData)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [14]:
goodAplModel.score(xData,yData)

0.4546184002639041

In [15]:
model = sm.OLS(yData,xData)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    apl   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.459
Method:                 Least Squares   F-statistic:                     51.93
Date:                Sat, 04 Mar 2017   Prob (F-statistic):           6.64e-17
Time:                        13:12:52   Log-Likelihood:                 150.39
No. Observations:                 120   AIC:                            -296.8
Df Residuals:                     118   BIC:                            -291.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Nasdaq         1.2373      0.135      9.183      0.0

In [16]:
months =  apl["Date"].apply(lambda x:int(str(x)[5:7]))[1:]
months[:5]

119    2
118    3
117    4
116    5
115    6
Name: Date, dtype: int64

In [17]:
dummy = sm.categorical(months.values.reshape(1,-1), drop=True)

In [18]:
dummy

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [19]:
xData = np.hstack((dummy, returns["Nasdaq"].values.reshape(len(returns["Nasdaq"]),-1)))
xData[:5]

array([[  0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         -1.93917974e-02],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          2.27220629e-03],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          4.27190662e-02],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.000

In [20]:
model = sm.OLS(yData,xData)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    apl   R-squared:                       0.524
Model:                            OLS   Adj. R-squared:                  0.471
Method:                 Least Squares   F-statistic:                     9.827
Date:                Sat, 04 Mar 2017   Prob (F-statistic):           1.13e-12
Time:                        13:13:37   Log-Likelihood:                 161.21
No. Observations:                 120   AIC:                            -296.4
Df Residuals:                     107   BIC:                            -260.2
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1            -0.0215      0.021     -1.014      0.3